# マルチンゲール

---

### 定義(マルチンゲール)

$X_1,X_2,...,X_n$は確率変数(独立ではない)とします．

そのおき以下の等式が成り立ちます．
$$
\mathbb{E}\left[X_{n+1} \mid X_1, X_2, \ldots, X_n\right]=X_n
$$

直感的な理解としては，条件付き期待値が直前の確率変数と，等しいことがわかります．

---

#### 使用例

強化学習でサンプル効率を求める際に集中不等式を取り扱う時があるのですが，その場合は，一般的にはHoeffidingやBernsteinの不等式を用います．

しかしながら，確率変数が互いに独立ではない時に．適用できません．

その時，マルチンゲールを考えて，集中不等式を解く方法があります．


---

## 解説

$\Omega$を測度空間として，pはその空間内の測度とします．$\mathcal{F}$をその測度空間の$\sigma$加法族とします．

関数$f: \Omega \rightarrow \mathbb{R}$は$\mathcal{F}$-可測とします．

$\mathcal{F}$-可測とは，$\forall B \in \mathcal{B}(\mathbb{R}): f^{-1}(B) \in \mathcal{F}$が成り立つ関数です．

ここで，$\mathcal{B}(\mathbb{R})$はボレル集合族です．(ボレル集合とはここでは実数直線上での開区間や閉区間，半開区間を含む最小の$\sigma$-加法族です)

なぜ，$\mathcal{F}$-可測のようなことを考えるかというと，一般的な統計学の考えで表すと，確率変数に変換するような操作です．(間違ってる認識かも?)

---
#### **期待値**
上記で説明した，概念を使い，期待値を定義すると，

$$
\mathrm{E}[f]=\mathrm{E}[f(x) \mid x \in \Omega]:=\sum_{x \in \Omega} f(x) p(x)
$$

---

#### 条件付き期待値


$\mathcal{F}$を$\Omega$上で定義されたシグマ加法族とします．

このシグマ加法族の上で条件付き期待値を取ると，
$$
\mathrm{E}[f \mid \mathcal{F}](x):=\frac{1}{\sum_{y \in \mathcal{F}(x)} p(y)} \sum_{y \in \mathcal{F}(x)} f(y) p(y)
$$

ここで，$\mathcal{F}(x)$は$x$を含む最小の$\mathcal{F}$の要素とします．

---

#### フィルター

フィルター$\mathbf{F}$とはシグマ加法族の部分集合に関する操作で生じる概念です．

以下に例を示します．

$$
\{0, \Omega\}=\mathcal{F}_0 \subset \mathcal{F}_1 \subset \cdots \subset \mathcal{F}_n=\mathcal{F} .
$$


ここで$X$をマルチンゲールとします．

各$\mathbf{F}$で関連づけられた確率変数を各々$X_1,X_2,...,X_n$とすると，

$$
X_i=\mathrm{E}\left[X \mid \mathcal{F}_i\right]
$$

が成り立ちます．

これが意味していることは確率変数$X_i$が現在持っている情報に基づいてしか，期待値を計算できないということです．


---

### Azuma's 不等式

マルチンゲール$\mathbf{X}$があり，それが$\mathbf{c}$-リプシッツ連続であるとすると，次の不等式が成り立つ．

$$
\operatorname{Pr}(|X-\mathrm{E}(X)| \geq \lambda) \leq 2 e^{-\frac{\lambda^2}{2 \sum_{i=1}^n c_i^2}}
$$

つまり，$\mathbf{c}$がわかっていれば，どれだけ期待値から離れているかの確率を計算できるわけですね．

コードでその確率を計算してみます．

In [9]:
import numpy as np

def azuma_inequality(c_values, lambda_value):
    sum_c_squared = np.sum(np.square(c_values))
    exponent = -lambda_value**2 / (2 * sum_c_squared)
    probability = 2 * np.exp(exponent)
    return probability


c_values = np.array([0.2, 0.3, 0.4, 0.5, 0.2])  # c_i の値
lambda_value = 1.5  # λの値

# Azumaの不等式を適用して確率を計算
probability = azuma_inequality(c_values, lambda_value)

# 結果の表示
print(f"Probability:{probability}")


Probability:0.28750702281690166


#### Azuma's 不等式の証明

関数$f(x) = e^{tx}$を考えます．(凸関数)

$|x| \leq c$のようなxを考えると次のような不等式が成り立ちます．($x$はマルチンゲール差分列が適用できそうですね.)

$$
e^{t x} \leq \frac{1}{2 c}\left(e^{t c}-e^{-t c}\right) x+\frac{1}{2}\left(e^{t c}+e^{-t c}\right)
$$

上記の不等式も使うと，
$$
\begin{aligned}
& \mathrm{E}\left(e^{t\left(X_i-X_{i-1}\right)} \mid \mathcal{F}_{i-1}\right) \\
& \leq \mathrm{E}\left(\frac{1}{2 c_i}\left(e^{t c_i}-e^{-t c_i}\right)\left(X_i-X_{i-1}\right)+\frac{1}{2}\left(e^{t c_i}+e^{-t c_i}\right) \mid \mathcal{F}_{i-1}\right) \\
&=\frac{1}{2}\left(e^{t c_i}+e^{-t c_i}\right) \\
& \leq e^{t^2 c_i^2 / 2}
\end{aligned}
$$

2項目から3項目の式の変換は$\mathrm{E}\left(X_i-X_{i-1} \mid \mathcal{F}_{i-1}\right)=0$を使っています．

これを書き換えると，$\mathrm{E}\left(e^{t X_i} \mid \mathcal{F}_{i-1}\right) \leq e^{t^2 c_i^2 / 2} e^{t X_{i-1}}$が成り立ちますね．

Azuma's 不等式の左辺では$\mathrm{E}\left(e^{t X}\right)$のバウンドが必要なので，解いていきます．

$$
\begin{aligned}
& \mathrm{E}\left(e^{t X}\right)=\mathrm{E}\left(\mathrm{E}\left(e^{t X_n} \mid \mathcal{F}_{n-1}\right)\right) \\
& \leq e^{t^2 c_n^2 / 2} \mathrm{E}\left(e^{t X_{n-1}}\right) \\
& \vdots \\
& \leq \prod_{i=1}^n e^{t^2 c_i^2 / 2} \mathrm{E}\left(e^{t X_0}\right) \\
&=e^{\frac{1}{2} t^2 \sum_{i=1}^n c_i^2} e^{t \mathrm{E}(X)} .
\end{aligned}
$$


これらを使い，Azuma's 不等式の左辺を書き換えながら右辺に近づけていきます．

$$
\begin{aligned}
\operatorname{Pr}(X \geq \mathrm{E}(X)+\lambda) & =\operatorname{Pr}\left(e^{t(X-\mathrm{E}(X))} \geq e^{t \lambda}\right) \\
& \leq e^{-t \lambda} \mathrm{E}\left(e^{t(X-\mathrm{E}(X))}\right) \\
& \leq e^{-t \lambda} e^{\frac{1}{2} t^2 \sum_{i=1}^n c_i^2} \\
& =e^{-t \lambda+\frac{1}{2} t^2 \sum_{i=1}^n c_i^2}
\end{aligned}
$$

こで，tについての二次関数を考え，最小のtを求めると，$t=\frac{\lambda}{\sum_{i=1}^n c_i^2}$となり，書き換えると，

$$
\begin{aligned}
\operatorname{Pr}(X \geq \mathrm{E}(X)+\lambda) & \leq e^{-t \lambda+\frac{1}{2} t^2 \sum_{i=1}^n c_i^2} \\
& =e^{-\frac{\lambda^2}{2 \sum_{i=1}^n c_i^2}} .
\end{aligned}
$$

示せましたね．





---

マルチンゲールの例

$Y_1,Y_2,...,Y_n$があり，$X = Y_1 + Y_2 + ...,+ Y_n$とする．

各$Y_n$はそれぞれシグマ加法族$\mathcal{F_i}$で作られたものとする．

このシグマ加法族に対して，次のようなフィルターがあるとすると，
$$
\{0, \Omega\}=\mathcal{F}_0 \subset \mathcal{F}_1 \subset \cdots \subset \mathcal{F}_n=\mathcal{F}
$$

各$X_i$は次のように書けるので，$X_i=\sum_{j=1}^i Y_j+\sum_{j=i+1}^n \mathrm{E}\left(Y_j\right)$，マルチンゲールとなる．

---

### 劣マルチンゲール(sub-martingale) と優マルチンゲール(super-martingale)

まず，二つの定義から説明すると，劣マルチンゲールは$\mathrm{E}\left(X_i \mid \mathcal{F}_{i-1}\right) \leq X_{i-1}$, 優マルチンゲールは$\mathrm{E}\left(X_i \mid \mathcal{F}_{i-1}\right) \geq X_{i-1}$となるものである．

劣マルチンゲールでは次のようなことが成り立つとある不等式が成り立つとされる．．

Xを劣マルチンゲールとして，$\phi_i$を$X_{i-1}$に関する線形関数とする．

$$
\operatorname{Var}\left(X_i \mid \mathcal{F}_{i-1}\right) \leq \phi_i X_{i-1}
$$

Mをある定数とすると，

$$
X_i-\mathrm{E}\left(X_i \mid \mathcal{F}_{i-1}\right) \leq M
$$

この２つが成り立つ時，次の不等式が成り立つ．

$$
\operatorname{Pr}\left(X_n \geq X_0+\lambda\right) \leq e^{-\frac{\lambda^2}{2\left(\left(X_0+\lambda\right)\left(\sum_{i=1}^\lambda \phi_i\right)+M \lambda / 3\right)}} .
$$

次にこの拡張したものを考える．

$$
\operatorname{Var}\left(X_i \mid \mathcal{F}_{i-1}\right) \leq \sigma_i^2+\phi_i X_{i-1}
$$

$$
X_i-\mathrm{E}\left(X_i \mid \mathcal{F}_{i-1}\right) \leq a_i+M
$$

とすると，

$$
\operatorname{Pr}\left(X_n \geq X_0+\lambda\right) \leq e^{-\frac{1}{2\left(\sum_{i=1}^n\left(\sigma_i^2+a_i^2\right)+\left(X_0+\lambda\right)\left(\sum_{i=1}^n \phi_i\right)+M \lambda / 3\right)}} .
$$

$a , \sigma, \phi, M$は正の定数とします．


これは$\sigma,a$を0にすれば先ほど示した不等式と同じ条件になりますね．

---

#### 証明

$\operatorname{Pr}\left(X_n \geq X_0+\lambda\right) \leq e^{-\frac{1}{2\left(\sum_{i=1}^n\left(\sigma_i^2+a_i^2\right)+\left(X_0+\lambda\right)\left(\sum_{i=1}^n \phi_i\right)+M \lambda / 3\right)}}$.の証明をしていきます．


 マルチンゲールの期待値のモーメントを考えます．

 $$
\begin{aligned}
\mathrm{E}\left(e^{t X_i} \mid \mathcal{F}_{i-1}\right) & =e^{t \mathrm{E}\left(X_i \mid \mathcal{F}_{i-1}\right)+t a_i} \mathrm{E}\left(e^{t\left(X_i-\mathrm{E}\left(X_i \mid \mathcal{F}_{i-1}\right)-a_i\right)} \mid \mathcal{F}_{i-1}\right) \\
& =e^{t \mathrm{E}\left(X_i \mid \mathcal{F}_{i-1}\right)+t a_i} \sum_{k=0}^{\infty} \frac{t^k}{k !} \mathrm{E}\left(\left(X_i-\mathrm{E}\left(X_i \mid \mathcal{F}_{i-1}\right)-a_i\right)^k \mid \mathcal{F}_{i-1}\right) \\
& \leq e^{t \mathrm{E}\left(X_i \mid \mathcal{F}_{i-1}\right)+\sum_{k=2}^{\infty} \frac{t^k}{k !} \mathrm{E}\left(\left(X_i-\mathrm{E}\left(X_i \mid \mathcal{F}_{i-1}\right)-a_i\right)^k \mid \mathcal{F}_{i-1}\right)}
\end{aligned}
$$

途中でexp関数のテーラー展開を考えています．

式の２項目を書き直していきます．

$X_i-\mathrm{E}\left(X_i \mid \mathcal{F}_{i-1}\right)-a_i \leq M$から，
$$
\begin{aligned}
& \sum_{k=2}^{\infty} \frac{t^k}{k !} \mathrm{E}\left(\left(X_i-\mathrm{E}\left(X_i \mid \mathcal{F}_{i-1}\right)-a_i\right)^k \mid \mathcal{F}_{i-1}\right) \\
& \leq \frac{g(t M)}{2} t^2 \mathrm{E}\left(\left(X_i-\mathrm{E}\left(X_i \mid \mathcal{F}_{i-1}\right)-a_i\right)^2 \mid \mathcal{F}_{i-1}\right) \\
&=\frac{g(t M)}{2} t^2\left(\operatorname{Var}\left(X_i \mid \mathcal{F}_{i-1}\right)+a_i^2\right) \\
& \leq \frac{g(t M)}{2} t^2\left(\sigma_i^2+\phi_i X_{i-1}+a_i^2\right)
\end{aligned}
$$

で，この式を使い，最初の式を書き直すと，

$$
\begin{aligned}
\mathrm{E}\left(e^{t X_i} \mid \mathcal{F}_{i-1}\right) & \leq e^{t \mathrm{E}\left(X_i \mid \mathcal{F}_{i-1}\right)+\sum_{k=2}^{\infty} \frac{t^k}{k !} \mathrm{E}\left(\left(X_i-\mathrm{E}\left(X_i \mid \mathcal{F}_{i-1}-\right)-a_i\right)^k \mid \mathcal{F}_{i-1}\right)} \\
& \leq e^{t X_{i-1}+\frac{g(t M)}{2} t^2\left(\sigma_i^2+\phi_i X_{i-1}+a_i^2\right)} \\
& =e^{\left(t+\frac{g(t M)}{2} \phi_i t^2\right) X_{i-1}} e^{\frac{t^2}{2} g(t M)\left(\sigma_i^2+a_i^2\right)}
\end{aligned}
$$